# Noodling with ideas from On Numbers and Games

The notation introduced in On Numbers, brief demo for my future usage:

In [1]:
(require '[clojupyter.misc.display :as display])
(display/markdown "\\begin{equation}
\\{ \\emptyset | \\emptyset \\}
\\end{equation}")

\begin{equation}
\{ \emptyset | \emptyset \}
\end{equation}

Need to come up with a manageable way not to have to do all escape characters all the time.

\begin{equation}
\{\emptyset|\emptyset\}
\end{equation}

In [2]:
(defn simple-conway-notation [n1 n2] (str "\\begin{equation} \\{" n1 "|" n2 "\\}\\end{equation}"))

#'user/simple-conway-notation

In [3]:
(display/markdown (simple-conway-notation "\\emptyset" "\\emptyset"))

\begin{equation} \{\emptyset|\emptyset\}\end{equation}

In [4]:
(display/markdown (simple-conway-notation 0 1))

\begin{equation} \{0|1\}\end{equation}

## Is it a number?
Conway says that if an expression contains only numbers on the left and the right and if there is no number on the left equal to or greater than any number on the left, then the resulting expression is considered equal to a number.  Let's assume it's ok for now to represent the notation from the book as two Clojure vectors.

In [24]:
(defn is-number [left right]
    (if (= 0 (count right))
           true
           (let [least-r (reduce min right)
                 greatest-l (reduce max left)]
               (> least-r greatest-l))))

#'user/is-number

In [25]:
(is-number [1 2 3] [4 5 6])

true

In [26]:
(is-number [1 2 3] [])

true

In [27]:
(is-number [2] [1])

false

A simple case of two nonempty sets that fit the rule, then an empty set for the right that can only pass, then an example of something that is not a number.  A minor improvement must be made to the notation code to display some of these scenarios:

In [31]:
(defn simple-conway-notation [n1 n2]
    (str "\\begin{equation} \\{"
         (clojure.string/join "," n1)
         "|"
         (clojure.string/join "," n2)
         "\\}\\end{equation}"))

#'user/simple-conway-notation

In [34]:
(defn ask-is-num [left right]
    (str (simple-conway-notation left right) " "
         (if (is-number left right) "is" "is not")
         " a number"))

#'user/ask-is-num

In [38]:
(display/markdown (str
                   (ask-is-num [1 2 3] [4 5 6])
                   (ask-is-num [1 2 3] [])
                   (ask-is-num [2] [1])))

\begin{equation} \{1,2,3|4,5,6\}\end{equation} is a number\begin{equation} \{1,2,3|\}\end{equation} is a number\begin{equation} \{2|1\}\end{equation} is not a number

## Greater than or equal
From the book, to test $x \ge y$ we will need the right side of the expression that makes up x and the left side of the expression that makes up y.  We're still assuming each instance of this notation can be represented by two Clojure vectors.  The problem now is that the definition for $x \ge y$ is self referential.  There is a base case because of how the rule is phrased and because of the existence of the null set.  Here's an implementation of the base case:

In [50]:
(defn x-gt-or-eq-y [[x-left x-right] [y-left y-right]]
    (let [x-r-empty (empty? x-right)
          y-l-empty (empty? y-left)]
        (if (and x-r-empty y-l-empty) true "shrug")))

#'user/x-gt-or-eq-y

In [49]:
(x-gt-or-eq-y [[1] [3]] [[-1] [1]])

shrug

The translation here is the question as to whether $\{1|3\}=2 \ge \{-1|1\} = 0$ which is true but cannot be evaluated by the base case because none of the sets in this case are empty.

In [51]:
(x-gt-or-eq-y [[1 2 3] []] [[] [1 2 3]])

true

Translation: $\{1,2,3|\}= 4\stackrel{?}{\ge}\{|1,2,3\}= 0$

This is easy for the base case code.  The recursive case forces me to stop using actual Clojure numbers because I want to keep the implementation in terms of the set notations.  Really this is all I was actually doing when I was only using whether the right of x is empty or the left of y is empty, so at this point strictly speaking the above, the input that led to a "shrug" in the earlier step is invalid and really so is the one immediately above.